In [1]:
%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer


con=sqlite3.connect('database.sqlite') 

filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 limit 1000
""", con) 


def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative



In [2]:
filtered_data.shape #looking at the number of attributes and size of the data
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [7]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [8]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(998, 10)

In [9]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

99.8

In [10]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [11]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [12]:
#BoW
count_vect = CountVectorizer() #in scikit-learn
final_counts = count_vect.fit_transform(final['Text'].values)


In [14]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(998, 10)


positive    841
negative    157
Name: Score, dtype: int64

In [15]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [16]:
final_counts.get_shape()

(998, 6033)

In [17]:
print(final)

       Id   ProductId          UserId  \
10     11  B0001PB9FE  A3HDKO7OW0QNK4   
232   253  B0002567IW  A1SSKFPX72MSMR   
171   188  B00029XIZI  A2S72TUJDQUBMH   
932  1011  B0002MKFEM  A3QLX72AO0DD5Z   
893   969  B0002XIB2Y  A3NV17B17PFB7G   
894   970  B0002XIB2Y  A1O3WOX0KBBV3W   
895   971  B0002XIB2Y  A2DRQBOGCTC5HP   
896   972  B0002XIB2Y  A1891H0TIXLDXA   
380   414  B00061KYVI  A29ZSRDOYVO735   
379   413  B00061KYVI  A3JE18D4XD2PYF   
378   412  B00061KYVI  A28RMYGO9RGA0K   
319   351  B00067AD4U   A7DSY3M6P6RG3   
316   348  B00067AD4U  A2OB1BFAEPAYBM   
318   350  B00067AD4U  A2LYOWC3FC73XK   
317   349  B00067AD4U  A3GT3KCGY9EYF4   
791   856  B0007NG568  A1XDNMQMY5PYFZ   
790   855  B0007NG568  A2BF7B1EJBMNSY   
789   854  B0007NG568   A8AJFY7BX21SS   
788   853  B0007NG568   AEFA52M4MT4H6   
787   852  B0007NG568  A2YPBLSWGVCGVL   
766   828  B0007NG56I  A1HHS1SELWW6NY   
767   829  B0007NG56I   AMQ8L5JWWDGIA   
768   830  B0007NG56I   AN808YPCW2C2F   
11     12  B0009

In [14]:
print(final_counts)

  (0, 29981)	1
  (0, 112121)	1
  (0, 68341)	1
  (0, 47909)	1
  (0, 5093)	1
  (0, 19419)	1
  (0, 98814)	1
  (0, 112630)	1
  (0, 20386)	1
  (0, 104542)	2
  (0, 112660)	1
  (0, 7750)	1
  (0, 28971)	1
  (0, 59142)	2
  (0, 74846)	1
  (0, 94431)	1
  (0, 58762)	1
  (0, 113360)	1
  (0, 72824)	1
  (0, 7296)	2
  (0, 65217)	1
  (0, 89722)	1
  (0, 39520)	1
  (0, 57417)	1
  (0, 111991)	1
  :	:
  (364170, 65220)	1
  (364170, 96057)	1
  (364170, 56471)	1
  (364170, 114561)	1
  (364170, 103466)	1
  (364170, 111267)	1
  (364170, 109946)	1
  (364170, 53384)	1
  (364170, 61424)	1
  (364170, 96374)	1
  (364170, 49495)	1
  (364170, 114517)	1
  (364170, 103941)	1
  (364170, 112937)	1
  (364170, 50620)	1
  (364170, 103443)	2
  (364170, 29981)	1
  (364170, 104542)	6
  (364170, 53557)	3
  (364170, 8302)	2
  (364170, 59284)	2
  (364170, 10401)	1
  (364170, 96473)	1
  (364170, 71724)	1
  (364170, 103749)	2


In [19]:
import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
##nltk.download('stopwords')
stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english')

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')   ### Replace any values in < > with '' 
    cleantext = re.sub(cleanr, ' ', sentence)  ## Sub means subsitute 
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

print(stop)
print('************************************')
print(sno.stem('tasty'))

{'hers', 'under', 'does', 'so', 'were', 'further', 's', 'being', 'i', 'didn', 'no', 'while', 'having', "mustn't", 'wasn', 'our', "aren't", 'from', 'if', 'such', 'than', 'when', 'should', 'did', 'myself', 'do', 'which', 'for', "you'd", 'whom', 'as', 'we', 'not', 'against', 'y', 'me', 'very', 'up', 'an', 'with', 'during', 'once', 'other', 'or', 'herself', 'how', "she's", 'their', 'couldn', 'what', 'mustn', 'between', 'nor', 'above', 'he', 'it', 'that', "isn't", 'before', 'yourselves', 'now', 'your', "couldn't", 'again', 'himself', 'was', 'o', 'out', 'they', 'all', 'at', 'ours', 'hadn', 'because', 'this', 'are', 'and', 'will', 'the', 'she', 'by', 'too', 'd', 'mightn', "weren't", 'hasn', 'him', 'yours', "wouldn't", 'have', "didn't", 'same', 'through', "hadn't", 'isn', 'theirs', "haven't", 'doing', "shouldn't", 'won', 'am', 'down', 'a', "should've", 'few', 'her', 'm', 'in', "you've", 'be', 'only', 'doesn', 'had', 'there', 'most', 'on', "don't", 'who', "you're", 'don', 'after', 'of', 'oursel

In [18]:
# find sentences containing HTML tags
import re
i=0;
for sent in final['Text'].values:
    if (len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
    i += 1;

0
I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!


In [13]:
final['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 

NameError: name 'final_string' is not defined

In [20]:
#Code for implementing step-by-step the checks mentioned in the pre-processing phase
# this code takes a while to run as it needs to run on 500k sentences.
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():    ### split words based on white spaces
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):     ## If alpha numeric 
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive': 
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

In [ ]:
#print (final_string)

In [21]:
final['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 

In [23]:
final.head(3) #below the processed review can be seen in the CleanedText Column 
final.head(3) #below the processed review can be seen in the CleanedText Column 


# store final table into an SQlLite table for future.
conn = sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn, flavor=None, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

In [24]:
#BoW
count_vect = CountVectorizer() #in scikit-learn
final_counts = count_vect.fit_transform(final['Text'].values)

In [25]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [26]:
final_counts.get_shape()

(998, 6033)

In [27]:
freq_dist_positive=nltk.FreqDist(all_positive_words)
freq_dist_negative=nltk.FreqDist(all_negative_words)
print("Most Common Positive Words : ",freq_dist_positive.most_common(1))  ## List Most common words (in the example 20 words)
print("Most Common Negative Words : ",freq_dist_negative.most_common(20))

Most Common Positive Words :  [(b'chip', 508)]
Most Common Negative Words :  [(b'chip', 106), (b'tast', 97), (b'like', 91), (b'product', 68), (b'bag', 63), (b'one', 56), (b'food', 53), (b'flavor', 52), (b'kettl', 46), (b'use', 46), (b'tri', 43), (b'would', 43), (b'good', 41), (b'box', 40), (b'eat', 38), (b'brand', 36), (b'look', 35), (b'dont', 34), (b'buy', 34), (b'amazon', 33)]


In [28]:
count_vect = CountVectorizer(ngram_range=(1,2) ) #in scikit-learn (unigram and < bigram)
final_bigram_counts = count_vect.fit_transform(final['Text'].values)

In [29]:
final_bigram_counts.get_shape()

(998, 43815)

In [30]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)

/root/anaconda2/envs/py34/lib/python3.4/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [32]:
final_tf_idf

<998x43815 sparse matrix of type '<class 'numpy.float64'>'
	with 115655 stored elements in Compressed Sparse Row format>

In [31]:
final_tf_idf.get_shape()

(998, 43815)

In [34]:
features = tf_idf_vect.get_feature_names()
len(features)

43815

In [40]:
features[1001:1019]

['after hunting',
 'after lapse',
 'after lifetime',
 'after long',
 'after longer',
 'after looking',
 'after lunch',
 'after many',
 'after melting',
 'after months',
 'after my',
 'after noticed',
 'after once',
 'after opening',
 'after reading',
 'after school',
 'after several',
 'after short']

In [47]:
# covnert a row in saprsematrix to a numpy array
print(final_tf_idf[3,:].toarray()[0])  ## Vector and numpy array 

[0. 0. 0. ... 0. 0. 0.]


In [51]:
# source: https://buhrmann.github.io/tfidf-analysis.html
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

top_tfidf = top_tfidf_feats(final_tf_idf[2,:].toarray()[0],features,5)  ## for review 2 vetorr 2 and convrting to numpy array and showing top 5 features

In [52]:
top_tfidf

,feature,tfidf
0,spots,0.298478
1,hot spots,0.298478
2,hot,0.181066
3,it tolerablel,0.165103
4,were full,0.165103


In [ ]:
# Using Google News Word2Vectors
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

# in this project we are using a pretrained model by google
# its 3.3G file, once you load this into your memory 
# it occupies ~9Gb, so please do this step only if you have >12G of ram
# we will provide a pickle file wich contains a dict , 
# and it contains all our courpus words as keys and  model[word] as values
# To use this code-snippet, download "GoogleNews-vectors-negative300.bin" 
# from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# it's 1.9GB in size.

### Creates dense vector - Maximum similarity 1 and min zero

#model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
